# Anthropic Claude Function Calling (Tools) Example
This notebook demonstrates how to use function calling with Anthropic in Dinnovos Agent.

In [ ]:
import os
import json
from dinnovos.llms.anthropic import AnthropicLLM

## 1. Configure API Key

In [ ]:
# Configure your OpenAI API key
api_key = os.getenv("ANTHROPIC_API_KEY") or "your-api-key-here"

In [ ]:
# Create LLM instance
llm = AnthropicLLM(api_key=api_key, model="claude-sonnet-4-20250514")

## 2. Define Tools (Functions)

In [ ]:
# Define available tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Bogotá, Colombia"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform a mathematical calculation",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate, e.g. '2 + 2'"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

## 3. Implement the Functions

In [ ]:
def get_weather(location: str, unit: str = "celsius") -> dict:
    """Mock weather function"""
    return {
        "location": location,
        "temperature": 22 if unit == "celsius" else 72,
        "unit": unit,
        "condition": "Sunny",
        "humidity": 65
    }

def calculate(expression: str) -> dict:
    """Calculator function"""
    try:
        result = eval(expression)
        return {"expression": expression, "result": result}
    except Exception as e:
        return {"error": str(e)}

# Dictionary of available functions
available_functions = {
    "get_weather": get_weather,
    "calculate": calculate
}

---

# 🚀 RECOMMENDED METHOD: call_with_function_execution()

**The easiest way** to use function calling. Automatically handles the entire cycle.

## Example 1: Basic Usage (Automatic)

In [ ]:
# One call does it all!
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá?"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True  # See the complete process
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])
print(f"\n🔄 Iterations: {result['iterations']}")
print(f"🔧 Functions called: {len(result['function_calls'])}")

## Example 2: Multiple Functions

In [ ]:
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Compare the weather in Bogotá and Medellín, and calculate 25 * 4"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])
print(f"\n🔄 Iterations: {result['iterations']}")
print(f"🔧 Total functions called: {len(result['function_calls'])}")

## Example 3: Silent Mode

In [ ]:
# Without verbose - just get the result
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Calculate (100 + 50) / 3"}],
    tools=tools,
    available_functions=available_functions,
    verbose=False
)

print("Response:", result['content'])

## Example 2: Multiple Functions

In [ ]:
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Compare the weather in Bogotá and Medellín, and calculate 25 * 4"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])
print(f"\n🔄 Iterations: {result['iterations']}")
print(f"🔧 Total functions called: {len(result['function_calls'])}")

## Example 3: Silent Mode

In [ ]:
# Without verbose - just get the result
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Calculate (100 + 50) / 3"}],
    tools=tools,
    available_functions=available_functions,
    verbose=False
)

print("Response:", result['content'])

## Example 4: View Complete History

In [ ]:
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "What's the temperature in Cali?"}],
    tools=tools,
    available_functions=available_functions
)

print("\n📊 COMPLETE ANALYSIS:")
print(f"\n✅ Response: {result['content']}")
print(f"\n🔄 Iterations performed: {result['iterations']}")
print(f"\n🔧 Functions executed:")
for i, func_call in enumerate(result['function_calls'], 1):
    print(f"\n  {i}. {func_call['name']}")
    print(f"     Args: {func_call['arguments']}")
    print(f"     Result: {func_call['result']}")

## Example 5: Streaming with Function Execution

In [ ]:
print("=" * 70)
print("STREAMING WITH AUTOMATIC FUNCTION EXECUTION")
print("=" * 70)

for chunk in llm.call_stream_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá?"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True  # This will show debug info
):
    chunk_type = chunk.get("type")
    
    if chunk_type == "iteration_start":
        print(f"\n[Iteration {chunk.get('iteration')}]")
    
    elif chunk_type == "text_delta":
        # Stream text as it arrives
        print(chunk.get("content"), end="", flush=True)
    
    elif chunk_type == "function_call_start":
        # Function is being called
        print(f"\n🔧 Calling: {chunk.get('function_name')}")
        print(f"   Args: {chunk.get('arguments')}")
    
    elif chunk_type == "function_call_result":
        # Function completed
        print(f"✅ Result: {chunk.get('result')}")
    
    elif chunk_type == "final":
        # Final response
        print(f"\n\n{'='*70}")
        print(f"Completed in {chunk.get('iterations')} iterations")
        print(f"Functions called: {len(chunk.get('function_calls', []))}")
    
    elif chunk_type == "error":
        print(f"\n❌ Error: {chunk.get('content')}")

## Example 6: Streaming with Function Execution silently

In [ ]:
# Streaming with Function Execution silently
for chunk in llm.call_stream_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá?"}],
    tools=tools,
    available_functions=available_functions,
    verbose=False  # Silent mode - no debug info in console
):
    chunk_type = chunk.get("type")
    
    if chunk_type == "iteration_start":
        print(f"\n[Iteration {chunk.get('iteration')}]")
    
    elif chunk_type == "text_delta":
        # Stream text as it arrives
        print(chunk.get("content"), end="", flush=True)
    
    elif chunk_type == "function_call_start":
        # Function is being called
        print(f"\n🔧 Calling: {chunk.get('function_name')}")
        print(f"   Args: {chunk.get('arguments')}")
    
    elif chunk_type == "function_call_result":
        # Function completed
        print(f"   ✅ Result: {chunk.get('result')}")
    
    elif chunk_type == "final":
        # Final response
        print(f"\n\n{'='*70}")
        print(f"Completed in {chunk.get('iterations')} iterations")
        print(f"Functions called: {len(chunk.get('function_calls', []))}")
    
    elif chunk_type == "error":
        print(f"\n❌ Error: {chunk.get('content')}")

## Example 6: Streaming with Multiple Functions

Test streaming with a more complex query that requires multiple function calls.

In [ ]:
print("=" * 70)
print("STREAMING: Multiple Functions")
print("=" * 70)

for chunk in llm.call_stream_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá and calculate 15 * 7"}],
    tools=tools,
    available_functions=available_functions,
    verbose=False  # Set to True to see debug info
):
    chunk_type = chunk.get("type")
    
    if chunk_type == "iteration_start":
        iteration = chunk.get('iteration')
        if iteration > 1:
            print(f"\n\n[Iteration {iteration}]")
    
    elif chunk_type == "text_delta":
        print(chunk.get("content"), end="", flush=True)
    
    elif chunk_type == "function_call_start":
        print(f"\n\n🔧 Calling: {chunk.get('function_name')}")
        print(f"   Args: {chunk.get('arguments')}")
    
    elif chunk_type == "function_call_result":
        print(f"✅ Result: {chunk.get('result')}")
    
    elif chunk_type == "final":
        print(f"\n\n{'='*70}")
        print(f"✅ Completed in {chunk.get('iterations')} iterations")
        
        for i, func_call in enumerate(chunk.get('function_calls', []), 1):
            print(f"\n{i}. {func_call['name']}: {func_call['result']}")
    
    elif chunk_type == "error":
        print(f"\n❌ Error: {chunk.get('content')}")